In [ ]:
import numpy as np

In [ ]:
import pickle

## Process the data with the sampler and segmenter
with open('Data\\train_inputs.pkl', 'rb') as handle:
    list_of_trains = pickle.load(handle)

In [ ]:
list_of_trains[2]

In [ ]:
list_of_trains[0].T[0]

In [ ]:
dict_of_features_timeseries_len = dict()
for item in list_of_trains:
    for feature_idx, feature_vector in enumerate(item.T):
        if not feature_idx in dict_of_features_timeseries_len:
            dict_of_features_timeseries_len[feature_idx] = []
        dict_of_features_timeseries_len[feature_idx].append(len(feature_vector))

In [ ]:
import numpy as np
for feature_idx, count_timeseries_steps in dict_of_features_timeseries_len.items():
    print(count_timeseries_steps)
    print("CC channel: {} | Mean length of timeseries: {} | Max len: {} | Min len: {}".format(feature_idx, np.mean(count_timeseries_steps), np.max(count_timeseries_steps), np.min(count_timeseries_steps)))

In [ ]:
import matplotlib.pyplot as plt

counts = np.bincount(count_timeseries_steps)

plt.hist(count_timeseries_steps, bins='auto', color='purple',
                            alpha=0.9, rwidth=0.85)
plt.grid(axis='y', alpha=1)
plt.xlabel('Length of time-series for CC #12')
plt.ylabel('Frequency')
#plt.title('Distribution of length of time-series for CC #12 for the training data')

In [ ]:
## METHOD FOR MISSING VALUES BELOW

In [ ]:
## Test visuzalization before processing
for count, item in enumerate(list_of_trains):
    for feature_idx, feature_vector in enumerate(item.T):
        if feature_idx == 0:
            plt.plot(feature_vector)
        break
    if count == 1:
        break

In [ ]:
import pickle

inside_dict_of_time_steps_values = dict.fromkeys(list(range(1,27)), 0)
dict_of_cc_dicts_list_of_timestep_values = dict.fromkeys(list(range(1,13)), 0)
## Creating a dictionary with key CC channel index and value another dictionary. The second dictionary is
## with key timestep index (1-26 as 26 is the max length of timestep in the dataset) and value a list. 
## In this list, we are appending every value for the corresponding CC channel and timestep index.
## This structure gives us the opportunity to compute mean/median/std etc for each CC channel and all values that
## belong to every timestep for every sample.
## Basically every list ([sample1_t_x, sample2_t_x..., sample270_t_x]) corresponds to one timestep, where sample1_t_x ## corresponds to the value for sample1 at timestep t_x. Now, having multiple lists, where each corresponds to a single timestep from 1 to 26 (26 is the max timestep in the dataset) can help us find missing values at certain timesteps.
# EXAMPLE: Let's imagine we have a sample with length of its timeseries = 9 and we want to compute the rest of the timesteps for CC channel 1. This means we need to fill values for timesteps from 10 to 26. We are starting an iterative process from 10 to 26, I example for timestep 10. We get the list at:
## list_t10 = dict_of_cc_dicts_list_of_timestep_values[1][10], 1 - CC channel, 10 - timestep index. Let's call this list "list_t10". We want to set a value for the sample for timestep 10 so one reasonable way to do so is to calculate the mean of the elements in "list_t10". Therefore, we are averaging over all values at timestep 10 for CC 1 for all samples that have a value there. We set the mean value as a timestep 10 for our sample. Repeat for every missing timestep value.
for count, item in enumerate(list_of_trains):
    for feature_idx, feature_vector in enumerate(item.T, 1):
        for time_step, cc_value in enumerate(feature_vector, 1):
            ## Create dict of CC keys (CC) and value a dict (X)
            if not isinstance(dict_of_cc_dicts_list_of_timestep_values[feature_idx], dict):
                dict_of_cc_dicts_list_of_timestep_values[feature_idx] = dict.fromkeys(list(range(1,27)), 0)
            ## Create list (L) as value inside the dict (X). In the list (L) we will append all values 
            ## corresponding to the timestep (key of X) and depending on the channel (key of CC). 
            if not isinstance(dict_of_cc_dicts_list_of_timestep_values[feature_idx][time_step], list):
                dict_of_cc_dicts_list_of_timestep_values[feature_idx][time_step] = list()
            ## Append value for the current CC and corresponding time step
            dict_of_cc_dicts_list_of_timestep_values[feature_idx][time_step].append(cc_value)

# Save as pickles
with open('Data\\dict_of_cc_dicts_list_of_timestep_values.pkl', 'wb') as handle:
    pickle.dump(dict_of_cc_dicts_list_of_timestep_values, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

## Loading dictionary so it can be seen how it looks
with open('Data\\dict_of_cc_dicts_list_of_timestep_values.pkl', 'rb') as handle:
    dict_of_cc_dicts_modified = pickle.load(handle)

In [ ]:
## List with values for CC channel=1 and timestep=20
## It can be observed that only 43 samples from the dataset have values for CC channel=1 at timstep=20.
## Indexing for CC channels range is: 1 to 12
## Indexing for timesteps range is: 1 to 26
print(dict_of_cc_dicts_modified[1][20])
print(len(dict_of_cc_dicts_modified[1][20]))

In [ ]:
## Processing the dataset with the class ProcessingMissingData
from processing_missing_data import ProcessingMissingData
import pickle

# Load train data

## Process the data with the sampler and segmenter
with open('Data\\train_inputs.pkl', 'rb') as handle:
    list_of_trains = pickle.load(handle)

processing_missing_data_obj = ProcessingMissingData()
list_of_processed_trains = processing_missing_data_obj.get_processed_dataset(list_of_data=list_of_trains, type='mean')

In [ ]:
## Comparison with the raw training data for the last sample which has only 9 timesteps.

print(len(list_of_trains[269]))
print(len(list_of_processed_trains[269]))

In [ ]:
import matplotlib.pyplot as plt

## Test visuzalization before processing
for count, item in enumerate(list_of_trains):
    for feature_idx, feature_vector in enumerate(item.T):
        if feature_idx == 0:
            plt.plot(feature_vector)
        break
    if count == 2:
        break

In [ ]:
import matplotlib.pyplot as plt

## Test visuzalization before processing
for count, item in enumerate(list_of_processed_trains):
    for feature_idx, feature_vector in enumerate(item.T):
        if feature_idx == 0:
            plt.plot(feature_vector)
        break
    if count == 2:
        break

In [ ]:
import numpy as np

CC_channel1 = 1
CC_channel2 = 2
timestep_ = 21

## 21 here equals to 20 on the plot as of indexing starts from 0
print("Mean value for CC channel {} and timestep {}: {}".format(CC_channel1, timestep_, np.mean(dict_of_cc_dicts_modified[CC_channel1][timestep_])))
print("Mean value for CC channel {} and timestep {}: {}".format(CC_channel2, timestep_, np.mean(dict_of_cc_dicts_modified[CC_channel2][timestep_])))

In [ ]:
for cc_channel_key in dict_of_cc_dicts_modified:
    mod_dict = dict_of_cc_dicts_modified[cc_channel_key]
    for timestep_key, list_values in mod_dict.items():
        print("Timestep: {} | Values count: {}".format(timestep_key, len(list_values)))
    break

In [ ]:
mod_dict